<a href="https://colab.research.google.com/github/kangwonlee/eng-math-2/blob/main/Ch12_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [ ]:
import dataclasses

import matplotlib.pyplot as plt
import numpy as np
import sympy as sym



### Example 12.3.1



Symbolic



In [ ]:
x, p = sym.symbols('x p')
n = sym.symbols('n', integer=True)

bn = (2 / p) * sym.integrate(
    x * sym.sin((n*sym.pi/p)*x),
    (x, 0, p)
)
bn



In [ ]:
mx = 5

fig = plt.figure(figsize=(6, mx*2))

for n in range(1, mx):
    plt.subplot(mx, 1, n)

    x = np.linspace(0, 2+1e-7, 361)
    y = np.cos((n * np.pi * 0.5) * x)
    plt.fill_between(x, y, label=fr'$\cos \frac{{{n}}}{{2}}\pi x$')

    plt.xlabel('$x$')
    plt.legend(loc=0)
    plt.xlim(0, 2)
    plt.grid(True)

plt.tight_layout()



Numerical



In [ ]:
@dataclasses.dataclass
class FourierSeriesPlotter:
    N:int = 10
    p:float = 2.0
    def __post_init__(self):
        self.pinv = 1.0 / self.p

    def calc_f(self, x_rad):
        result = np.zeros_like(x_rad)
        x_phase = x_rad % (2 * self.p)
        result[x_phase<self.p] = x_phase[x_phase<self.p]
        result[x_phase>=self.p] = x_phase[x_phase>=self.p] - (2*self.p)
        return result

    def calc_an(self, n):
        return 0.0

    def calc_bn(self, n):
        return (self.p**2) * (-1)**(n+1) /(n * np.pi)

    def calc_mode_n(self, x, n):
        bn = self.calc_bn(n)
        n_pi_pinv = n*np.pi* self.pinv
        x_inside = n_pi_pinv * x
        mode_n = bn * np.sin(x_inside)
        return mode_n

    def calc_Sn(self, n_sigma, x):
        result = np.zeros_like(x)
        for n in range(1, n_sigma+1):
            mode_n = self.calc_mode_n(x, n)
            result += mode_n
        return result

    def plot(self):
        x = np.arange(-(self.p * 5), (self.p * 5)+1e-7, 0.01)
        x_vlines = np.arange(x.min(), x.max()+1e-7, self.p)

        fig, axs = plt.subplots(self.N-1, 2, figsize=(12, 1.5*(self.N-1)))

        axs_mode = axs[:,0]
        axs_fourier = axs[:,1]

        for n_sigma, ax_mode, ax_fourier in zip(range(1, self.N), axs_mode, axs_fourier):
            ax_mode.plot(x, self.calc_mode_n(x, n_sigma))

            ax_fourier.plot(x, self.calc_f(x))
            ax_fourier.vlines(x_vlines, -2, 2, linestyles='dotted')
            y_fourier = self.calc_Sn(n_sigma, x)
            ax_fourier.plot(x, y_fourier)

            ax_mode.set_xlim(x.min(), x.max())
            ax_mode.set_ylim(-2, 2)
            ax_fourier.set_xlim(x.min(), x.max())

            ax_mode.grid(True)
            ax_fourier.grid(True)

        plt.tight_layout()

    def calc_spectrum(self):
        freq_rad_list = [0]

        a_list = [self.calc_an(0)]
        b_list = [0]

        mag_list = [abs(self.calc_an(0))]
        arg_rad_list = [0.0]

        for n in range(1, self.N):
            an = self.calc_an(n)
            bn = self.calc_bn(n)

            freq_rad_list.append(n*np.pi/self.p)
            a_list.append(an)
            b_list.append(bn)

            mag_list.append((an**2 + bn**2)**0.5)
            arg_rad_list.append(np.arctan2(-bn, an))

        return {
            'freq(rad)': tuple(freq_rad_list),
            'real': tuple(a_list),
            'imag': tuple(b_list),
            'mag' : tuple(mag_list),
            'phase(rad)': tuple(arg_rad_list),
        }

    def plot_spectrum(self):
        s = self.calc_spectrum()

        freq_rad_list = s['freq(rad)']
        mag_list = s['mag']
        arg_rad_list = s['phase(rad)']

        # wrap phase to avoid excessive discontinuity
        arg_rad_array = np.array(arg_rad_list)
        arg_deg_array = np.rad2deg(arg_rad_list)

        freq_deg_array = np.array(freq_rad_list) * (0.5 / np.pi)

        plt.close()
        plt.subplot(2, 1, 1)
        plt.plot(freq_deg_array, mag_list, '.-')
        plt.ylabel('magnitude')
        plt.grid(True)

        plt.subplot(2, 1, 2)
        plt.plot(freq_deg_array, arg_deg_array, '.-')
        plt.xlabel('freq(Hz)')
        plt.ylabel('phase(deg)')
        plt.grid(True)
        plt.tight_layout()



In [ ]:
p = FourierSeriesPlotter()
p.plot()



### Spectrum



In [ ]:
p.plot_spectrum()



### Example 12.3.2



In [ ]:
class FourierSeriesPlotter_12_3_2(FourierSeriesPlotter):
    def calc_f(self, x_rad):
        result = np.ones_like(x_rad)
        x_phase = x_rad % (2 * self.p)
        result[x_phase>=np.pi] = -1.0
        return result

    def calc_bn(self, n):
        return 2.0 * (1-(-1)**(n)) /(n * np.pi)



In [ ]:
p = FourierSeriesPlotter_12_3_2(p=np.pi)
p.plot()



### Spectrum



In [ ]:
p.plot_spectrum()



## Example 12.3.4



$$
b_n
=
  2\int_{0}^{1}
    \pi t \sin n \pi t dt
$$



$$
x_p(t)
=
  \sum_{n=1}^{\infty}
    \frac{2(-1)^{n+1}}
      {n\left[k-mn^2\pi^2\right]}
    \sin n\pi t
$$



In [ ]:
n = sym.symbols('n', positive=True, integer=True)
t = sym.symbols('t')

bn = sym.simplify(
    2 * sym.integrate(
        sym.pi * t * sym.sin(n * sym.pi * t),
        (t, 0, 1)
    )
)
bn



In [ ]:
@dataclasses.dataclass
class FourierSeriesPlotter_12_3_4(FourierSeriesPlotter):
    m:float = 1.0 / 16.0
    k:float = 4.0

    def calc_f(self, x_rad):
        result = np.zeros_like(x_rad)
        x_phase = x_rad % (2 * self.p)
        result[x_phase<self.p] = np.pi * x_phase[x_phase<self.p]
        result[x_phase>=self.p] = np.pi * (x_phase[x_phase>=self.p] - (2*self.p))
        return result

    def calc_bn(self, n):
        return (
            (2 * (-1)**(n+1))
            /(n * (self.k - self.m * n * n * np.pi * np.pi))
        )

    def calc_mode_n(self, x, n):
        return self.calc_bn(n) * np.sin(n * np.pi * x)



Input vs output in time domain



In [ ]:
p = FourierSeriesPlotter_12_3_4(p=1)
p.plot()



Output spectrum



In [ ]:
p.plot_spectrum()

